In [1]:
# The goal is to find what neighborhoods have heterogeneous areas
# to decide which of them need special treatment

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from utils.load_data import load_data
from feature_engineering import engineer_features
from data_cleaning.data_cleaning import clean_data

In [5]:
train_df = load_data("data/train.csv")
train_df = engineer_features(train_df)
train_df = clean_data(train_df)

In [6]:
def calculate_neighborhood_entropy(df, features):
    """
    Calculate entropy for each feature within each neighborhood.
    
    Parameters:
    df: pandas DataFrame containing the data
    features: list of categorical features to analyze
    
    Returns:
    DataFrame with entropy scores for each feature in each neighborhood
    """
    results = []
    
    for neighborhood in df['Neighborhood'].unique():
        neighborhood_data = df[df['Neighborhood'] == neighborhood]
        
        # Calculate entropy for each feature
        feature_entropies = {}
        for feature in features:
            # Get value counts and calculate probabilities
            value_counts = neighborhood_data[feature].value_counts(normalize=True)
            # Calculate entropy
            feature_entropy = entropy(value_counts)
            feature_entropies[f'{feature}_entropy'] = feature_entropy
            
        # Add count of houses in neighborhood
        feature_entropies['house_count'] = len(neighborhood_data)
        
        # Create row for this neighborhood
        row = {'Neighborhood': neighborhood, **feature_entropies}
        results.append(row)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by total entropy (sum of all feature entropies)
    entropy_columns = [col for col in results_df.columns if col.endswith('_entropy')]
    results_df['total_entropy'] = results_df[entropy_columns].sum(axis=1)
    results_df = results_df.sort_values('total_entropy', ascending=False)
    
    return results_df

def analyze_heterogeneity(df):
    """
    Analyze neighborhood heterogeneity using multiple categorical features.
    """
    # Features to analyze
    categorical_features = [
        'MSSubClass',
        'BldgType',
        'HouseStyle',
        'OverallQual',
        'RoofStyle',
        'Exterior1st',
        'GarageType'
    ]
    
    # Calculate entropy
    entropy_df = calculate_neighborhood_entropy(df, categorical_features)
    
    # Print summary of most and least heterogeneous neighborhoods
    print("\nMost heterogeneous neighborhoods:")
    print(entropy_df.head()[['Neighborhood', 'total_entropy', 'house_count']].to_string())
    
    print("\nLeast heterogeneous neighborhoods:")
    print(entropy_df.tail()[['Neighborhood', 'total_entropy', 'house_count']].to_string())
    
    # Create detailed feature entropy breakdown
    feature_specific = entropy_df[['Neighborhood'] + [col for col in entropy_df.columns if col.endswith('_entropy')]]
    print("\nFeature-specific entropy breakdown for top 5 most heterogeneous neighborhoods:")
    print(feature_specific.head().to_string())
    
    return entropy_df

# Function to identify neighborhoods that might need special treatment
def identify_special_treatment_neighborhoods(entropy_df, entropy_threshold=None):
    """
    Identify neighborhoods that might need special treatment based on entropy scores.
    
    Parameters:
    entropy_df: DataFrame with entropy scores
    entropy_threshold: Optional threshold, if None will use mean + 1 std
    
    Returns:
    List of neighborhoods needing special treatment
    """
    if entropy_threshold is None:
        # Set threshold as mean + 1 standard deviation of total entropy
        entropy_threshold = (
            entropy_df['total_entropy'].mean() + 
            entropy_df['total_entropy'].std()
        )
    
    special_treatment = entropy_df[
        entropy_df['total_entropy'] > entropy_threshold
    ]['Neighborhood'].tolist()
    
    print(f"\nNeighborhoods needing special treatment (entropy > {entropy_threshold:.2f}):")
    for idx, neighborhood in enumerate(special_treatment, 1):
        print(f"{idx}. {neighborhood}")
    
    return special_treatment

In [7]:
# features = [
#     "MSSubClass",
#      "BldgType",
#      "HouseStyle",
#      "OverallQual",
#      "RoofStyle",
#      "Exterior1st",
#      "GarageType"
# ]
entropy_df = analyze_heterogeneity(train_df)
entropy_df


Most heterogeneous neighborhoods:
   Neighborhood  total_entropy  house_count
15      Edwards       9.610099          100
7       OldTown       8.722382          113
2       Crawfor       8.543410           51
13       IDOTRR       7.910597           37
4       Mitchel       7.648498           49

Least heterogeneous neighborhoods:
   Neighborhood  total_entropy  house_count
3       NoRidge       4.405948           41
20      NPkVill       2.359269            9
22       BrDale       2.162116           16
21      Blmngtn       1.742951           17
24      Blueste       0.693147            2

Feature-specific entropy breakdown for top 5 most heterogeneous neighborhoods:
   Neighborhood  MSSubClass_entropy  BldgType_entropy  HouseStyle_entropy  OverallQual_entropy  RoofStyle_entropy  Exterior1st_entropy  GarageType_entropy  total_entropy
15      Edwards            1.949826          0.735449            1.263441             1.522198           0.733991             2.041174            1.364

,Neighborhood,MSSubClass_entropy,BldgType_entropy,HouseStyle_entropy,OverallQual_entropy,RoofStyle_entropy,Exterior1st_entropy,GarageType_entropy,house_count,total_entropy
15,Edwards,1.949826,0.735449,1.263441,1.522198,0.733991,2.041174,1.364021,100,9.610099
7,OldTown,1.902442,0.493582,1.502813,1.568816,0.596108,1.771357,0.887265,113,8.722382
2,Crawfor,1.729404,0.481199,1.272335,1.396788,0.721290,1.904138,1.038256,51,8.543410
13,IDOTRR,1.637196,0.333011,1.255803,1.515961,0.516302,1.627390,1.024934,37,7.910597
4,Mitchel,1.640346,0.852268,1.119597,1.234393,0.540989,1.320537,0.940367,49,7.648498
11,NAmes,1.304921,0.346021,1.013206,1.198230,0.713222,1.911612,0.935498,225,7.422709
19,ClearCr,1.700433,0.000000,1.294391,1.301945,0.972451,1.586872,0.558648,28,7.414741
8,BrkSide,1.546457,0.149995,1.269277,1.511675,0.298758,1.608906,0.735856,58,7.120924
12,SawyerW,1.564915,0.643207,1.076360,1.329735,0.290210,1.376820,0.760386,59,7.041632
23,SWISU,1.385114,0.366925,1.204007,1.319556,0.167944,1.285956,0.970403,25,6.699905


In [8]:
identify_special_treatment_neighborhoods(entropy_df)


Neighborhoods needing special treatment (entropy > 8.11):
1. Edwards
2. OldTown
3. Crawfor


['Edwards', 'OldTown', 'Crawfor']

In [9]:
# entropy_df.to_string("nb_entropy.txt")

In [ ]:
# 1. Most heterogeneous neighborhoods (highest total entropy):
# - Edwards (9.61): Highest diversity in housing types, very high diversity in exterior materials
# - OldTown (8.72): Very diverse housing types and house styles
# - Crawfor (8.54): Well-balanced diversity across all features

# 2. Least heterogeneous neighborhoods (lowest total entropy):
# - Blueste (0.69): Almost no diversity, only BldgType shows any variation
# - Blmngtn (1.74): Very uniform, only slight variations in quality and roof style
# - BrDale (2.16): Uniform housing types but some variation in building types and garage types

# Key insights about neighborhoods needing special treatment:

# 1. Edwards (100 houses):
# - Highest MSSubClass entropy (1.95) - very diverse housing types
# - Highest Exterior1st entropy (2.04) - many different exterior materials
# - High GarageType entropy (1.36) - diverse garage configurations
# - Good sample size (100 houses)

# 2. OldTown (113 houses):
# - Second highest MSSubClass entropy (1.90)
# - Highest HouseStyle entropy (1.50)
# - Highest OverallQual entropy (1.57)
# - Largest sample size among special treatment group (113 houses)

# 3. Crawfor (51 houses):
# - High entropy across all features
# - Particularly high in Exterior1st (1.90)
# - Smaller sample size (51 houses) might make modeling more challenging

# Recommendations:
# 1. Consider separate models or neighborhood-specific features for these three areas
# 2. Pay special attention to house style and exterior features in these neighborhoods
# 3. Consider using interaction terms between neighborhood and key features
# 4. Might need more robust validation for Crawfor due to smaller sample size


In [ ]:
# Yes, that's a good approach! This way we can understand the unique characteristics and patterns in each heterogeneous neighborhood, while still having a solid understanding of the more homogeneous areas.

# Let's outline an EDA plan:

# 1. Edwards (100 houses):
# - Focus on diverse housing types (high MSSubClass entropy)
# - Analyze the variety of exterior materials
# - Study relationship between housing type and price
# - Look for any patterns in garage types
# - Check if quality varies significantly by housing type

# 2. OldTown (113 houses):
# - Deep dive into house styles and their price impact
# - Analyze quality variations
# - Study if there are price patterns by housing age/type
# - Check if certain combinations of features are more valuable

# 3. Crawfor (51 houses):
# - Careful analysis due to smaller sample size
# - Look for any outliers that might affect modeling
# - Study the exterior features and their price impact
# - Analyze if certain housing types command premium prices
# - Check for any unique feature combinations

# 4. Other Neighborhoods (Combined):
# - More standardized analysis
# - Focus on common patterns and relationships
# - Look for consistent price factors
# - Identify any outliers
# - Study if certain features have consistent impact across neighborhoods

# Would you like to start with any particular neighborhood, or should we create a structured approach to analyze them all systematically?

# Should we also create visual comparisons to see how these special neighborhoods differ from the more homogeneous ones?